<a href="https://colab.research.google.com/github/fazapramudya/Skripsi/blob/main/SPK_Pemilihan_Siswa_Olimpiade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SISTEM PENDUKUNG KEPUTUSAN PEMILIHAN SISWA OLIMPIADE**

## **Import Library yang digunakan untuk menganalisis data dan autentikasi ke Google Spreadsheet**

Dalam implementasi autentikasi bertujuan untuk mengkoneksikan antara Google Colab dengan Spreadsheet yang digunakan sehingga nilai yang terdapat dalam Spreadsheet dapat dilakukan perhitungan dan analisis.

In [ ]:
import pandas as pd # import library pandas untuk menganalisis data
from google.colab import auth
import gspread # import library gspread untuk mengautentikasi Google Colab dengan Spreadsheet
from google.auth import default
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
sht2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1WumkeZ_1niS4cQwdZzZLYk1c9AIaNrhB74dAsp2NGuI/edit#gid=397235510') # salin link spreadsheet yang akan digunakan
worksheet_name = 'Hasil'  # Ganti dengan nama sheet data nilai
worksheet = sht2.worksheet(worksheet_name)
worksheet_name2 = 'Pembobotan'  # Ganti dengan nama sheet pembobotan yang diinginkan
worksheet2 = sht2.worksheet(worksheet_name2)
# Baca data ke DataFrame
data = worksheet.get_all_values()
data2 = worksheet2.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])
df2 = pd.DataFrame(data2[1:], columns=data2[0])

## **Implementasi Pustaka pandas untuk Konversi Nilai**

Dalam implementasi pustaka pandas bertujuan untuk konversi nilai ini bertujuan untuk mengkonversi setiap kolom dalam DataFrame sehingga menjadi tipe data numerik.

In [ ]:
df['No'] = pd.to_numeric(df['No'], errors='coerce') # errors='coerce' berguna untuk  mengubah nilai non-numerik menjadi nilai NaN jika konversi tidak mungkin dilakukan.
df['nilaiHarian'] = pd.to_numeric(df['nilaiHarian'], errors='coerce')
df['nilaiUTS'] = pd.to_numeric(df['nilaiUTS'], errors='coerce')
df['MTK'] = pd.to_numeric(df['MTK'], errors='coerce')
df['fisika'] = pd.to_numeric(df['fisika'], errors='coerce')
df['kimia'] = pd.to_numeric(df['kimia'], errors='coerce')
df['biologi'] = pd.to_numeric(df['biologi'], errors='coerce')
df['ekonomi'] = pd.to_numeric(df['ekonomi'], errors='coerce')
df['geografi'] = pd.to_numeric(df['geografi'], errors='coerce')
df['TIK'] = pd.to_numeric(df['TIK'], errors='coerce')
df['kognitif'] = pd.to_numeric(df['kognitif'], errors='coerce')
df['afektif'] = pd.to_numeric(df['afektif'], errors='coerce')
df['psikomotorik'] = pd.to_numeric(df['psikomotorik'], errors='coerce')
df['evaluasi'] = pd.to_numeric(df['evaluasi'], errors='coerce')
df2['Bobot Kriteria'] = pd.to_numeric(df2['Bobot Kriteria'], errors='coerce')
df2['Bobot Sub-Kriteria'] = pd.to_numeric(df2['Bobot Sub-Kriteria'], errors='coerce')

## **Implementasi Matriks R**

Implementasi matriks R ini bertujuan untuk mencari nilai matriks R yang telah ternormalisasi dengan cara melakukan perhitungan pembagian antara masing-masing kriteria pada alternatif dengan nilai maksimum pada kriteria tersebut.

In [ ]:
# Mencari nilai terbesar
max_kognitif = df['kognitif'].max()
max_afektif = df['afektif'].max()
max_psikomotorik = df['psikomotorik'].max()

# Menampilkan nilai terbesar
print("Nilai Terbesar dalam Sub Kriteria: Kognitif = ", max_kognitif, ", Afektif = ", max_afektif,", Psikomotorik = ", max_psikomotorik)

Nilai Terbesar dalam Sub Kriteria: Kognitif =  70 , Afektif =  60 , Psikomotorik =  60


In [ ]:
# Mencari Nilai Matriks R pada setiap sub-kriteria tingkat evaluasi diri siswa
df['R_kognitif'] = df['kognitif'] / max_kognitif
df['R_afektif'] = df['afektif'] / max_afektif
df['R_psikomotorik'] = df['psikomotorik'] / max_psikomotorik

##**Implementasi Nilai Preferensi**

Implementasi nilai preferensi ini bertujuan untuk menjadi nilai preferensi dari setiap sub-kriteria yang ada dengan melakukan perhitungan pernjumlahan antara nilai matriks R masing-masing sub kriteria dikalikan dengan bobot masing-masing sub kriteria.

In [ ]:
bobotKriteria = 'Bobot Kriteria'  # Ganti dengan nama kolom yang diinginkan
bobotSub = 'Bobot Sub-Kriteria'  # Ganti dengan nama kolom yang diinginkan
k_kognitif= 0  # Ganti dengan nomor baris yang diinginkan
k_afektif= 1  # Ganti dengan nomor baris yang diinginkan
k_psikomotorik= 2  # Ganti dengan nomor baris yang diinginkan
k_UTS= 0  # Ganti dengan nomor baris yang diinginkan
k_olimpiade= 1  # Ganti dengan nomor baris yang diinginkan
k_evaluasi= 2  # Ganti dengan nomor baris yang diinginkan
k_harian= 3  # Ganti dengan nomor baris yang diinginkan

# Mengambil nilai bobot dari DataFrame
b_kognitif = df2.iloc[k_kognitif][bobotSub]
b_afektif = df2.iloc[k_afektif][bobotSub]
b_psikomotorik = df2.iloc[k_psikomotorik][bobotSub]
b_UTS = df2.iloc[k_UTS][bobotKriteria]
b_olimpiade = df2.iloc[k_olimpiade][bobotKriteria]
b_evaluasi = df2.iloc[k_evaluasi][bobotKriteria]
b_harian = df2.iloc[k_harian][bobotKriteria]

# Menghitung Nilai Preferensi
df['nilaiPreferensi'] = (df['R_kognitif'] * b_kognitif) + (df['R_afektif'] * b_afektif) + (df['R_psikomotorik'] * b_psikomotorik)

##**Implementasi Vektor S**

Implementasi vektor S ini dilakukan dengan cara melakukan perhitungan perkalian antara masing-masing kriteria dipangkatkan dengan bobot dari masing-masing kriteria yang telah ditentukan sebelumnya.

In [ ]:
# Menghitung Nilai Vektor S
df['S_MTK'] = (df['nilaiHarian'] ** b_harian) * (df['nilaiUTS'] ** b_UTS) * (df['MTK'] ** b_olimpiade) * (df['nilaiPreferensi'] ** b_evaluasi)
df['S_fisika'] = (df['nilaiHarian'] ** b_harian) * (df['nilaiUTS'] ** b_UTS) * (df['fisika'] ** b_olimpiade) * (df['nilaiPreferensi'] ** b_evaluasi)
df['S_kimia'] = (df['nilaiHarian'] ** b_harian) * (df['nilaiUTS'] ** b_UTS) * (df['kimia'] ** b_olimpiade) * (df['nilaiPreferensi'] ** b_evaluasi)
df['S_biologi'] = (df['nilaiHarian'] ** b_harian) * (df['nilaiUTS'] ** b_UTS) * (df['biologi'] ** b_olimpiade) * (df['nilaiPreferensi'] ** b_evaluasi)
df['S_ekonomi'] = (df['nilaiHarian'] ** b_harian) * (df['nilaiUTS'] ** b_UTS) * (df['ekonomi'] ** b_olimpiade) * (df['nilaiPreferensi'] ** b_evaluasi)
df['S_geografi'] = (df['nilaiHarian'] ** b_harian) * (df['nilaiUTS'] ** b_UTS) * (df['geografi'] ** b_olimpiade) * (df['nilaiPreferensi'] ** b_evaluasi)
df['S_TIK'] = (df['nilaiHarian'] ** b_harian) * (df['nilaiUTS'] ** b_UTS) * (df['TIK'] ** b_olimpiade) * (df['nilaiPreferensi'] ** b_evaluasi)

##**Implementasi Vektor V**

Implementasi Vektor V dilakukan dengan cara mencari total nilai dari setiap mata pelajaran lalu dilanjutkan dengan pembagian antara nilai vektor S dari masing-masing mata pelajaran dibagi dengan nilai total dari setiap mata pelajaran.

In [ ]:
# Menghitung total dari vektor S pada masing-masing mata pelajaran
MTKtotalS = df['S_MTK'].sum()
fisikatotalS = df['S_fisika'].sum()
kimiatotalS = df['S_kimia'].sum()
biologitotalS = df['S_biologi'].sum()
ekonomitotalS = df['S_ekonomi'].sum()
geografitotalS = df['S_geografi'].sum()
TIKtotalS = df['S_TIK'].sum()

# Menghitung Nilai Vektor V dari masing-masing mata pelajaran
df['V_MTK'] = df['S_MTK'] / MTKtotalS
df['V_fisika'] = df['S_fisika'] / fisikatotalS
df['V_kimia'] = df['S_kimia'] / kimiatotalS
df['V_biologi'] = df['S_biologi'] / biologitotalS
df['V_ekonomi'] = df['S_ekonomi'] / ekonomitotalS
df['V_geografi'] = df['S_geografi'] / geografitotalS
df['V_TIK'] = df['S_TIK'] / TIKtotalS

##**Implementasi Perhitungan  peringkat (ranking) untuk setiap kolom dalam DataFrame**

Implementasi perhitungan peringkat ini bertujuan untuk melakukan pengurutan terhadap nilai dari masing-masing nilai mata pelajaran dimulai dari nilai terbesar hingga terkecil.

In [ ]:
# Mengurutkan nilai vektor V siswa dari setiap mata pelajaran mulai dari yang terbesar hingga yang terkecil
df['peringkatMTK'] = df['V_MTK'].rank(ascending=False, method='dense').astype(int)
df['peringkatFisika'] = df['V_fisika'].rank(ascending=False, method='dense').astype(int)
df['peringkatKimia'] = df['V_kimia'].rank(ascending=False, method='dense').astype(int)
df['peringkatBiologi'] = df['V_biologi'].rank(ascending=False, method='dense').astype(int)
df['peringkatEkonomi'] = df['V_ekonomi'].rank(ascending=False, method='dense').astype(int)
df['peringkatGeografi'] = df['V_geografi'].rank(ascending=False, method='dense').astype(int)
df['peringkatTIK'] = df['V_TIK'].rank(ascending=False, method='dense').astype(int)

##**Implementasi Perangkingan masing-masing Mata Pelajaran**

Implementasi Perangkingan masing-masing mata pelajaran ini bertujuan untuk menampilkan dalam tabel 5 siswa terbaik dalam setiap mata pelajarannya yang nantinya dijadikan rekomendasi untuk menjadi kandidat siswa olimpiade

In [ ]:
# Mengurutkan DataFrame berdasarkan peringkat MTK
df_sortedMTK = df.sort_values(by='peringkatMTK', inplace = True)
kolom_mtk = ['Nama', 'Kelas' ,'peringkatMTK']
df_MTK = df.head(5)[kolom_mtk]
df_allMTK = df.head(97)[kolom_mtk]

# Menampilkan DataFrame setelah pengurutan
df_MTK

,Nama,Kelas,peringkatMTK
8,KEYSHA FATCHU ISHTAMAR,X A,1
37,SATRIA FATTAH PRAMONO,X B,2
2,ANINDITA SAVIRA,X A,3
19,VANNISA NAUFALIZZA PRAMETRI,X A,4
31,MUHAMMAD DJIBRIL AMURWABUMI ZULHAM,X B,5


In [ ]:
# Mengurutkan DataFrame berdasarkan peringkat Fisika
df_sortedFisika = df.sort_values(by='peringkatFisika', inplace = True)
kolom_fisika = ['Nama','Kelas', 'peringkatFisika']
df_fisika = df.head(5)[kolom_fisika]
df_allFisika = df.head(97)[kolom_fisika]

# Menampilkan DataFrame setelah pengurutan
df_fisika

,Nama,Kelas,peringkatFisika
0,AIRIN NURFAUZIAH MURSITO,X A,1
53,KALYCA KHANSA FADILLA,XI A,2
20,VIOLA AZIZAH,X A,3
39,ALEXA SERRAFONA JULIET,XI A,4
81,MUHAMMAD FAQIH ILMI,XI B,5


In [ ]:
# Mengurutkan DataFrame berdasarkan peringkat Kimia
df_sortedKimia = df.sort_values(by='peringkatKimia', inplace = True)
kolom_kimia = ['Nama', 'Kelas','peringkatKimia']
df_kimia = df.head(5)[kolom_kimia]
df_allKimia = df.head(97)[kolom_kimia]

# Menampilkan DataFrame setelah pengurutan
df_kimia

,Nama,Kelas,peringkatKimia
0,AIRIN NURFAUZIAH MURSITO,X A,1
53,KALYCA KHANSA FADILLA,XI A,2
20,VIOLA AZIZAH,X A,3
35,RAFI DZAKY UTAMA,X B,4
39,ALEXA SERRAFONA JULIET,XI A,5


In [ ]:
# Mengurutkan DataFrame berdasarkan peringkat Biologi
df_sortedBiologi = df.sort_values(by='peringkatBiologi', inplace = True)
kolom_biologi = ['Nama','Kelas', 'peringkatBiologi']
df_biologi = df.head(5)[kolom_biologi]
df_allBiologi = df.head(97)[kolom_biologi]

# Menampilkan DataFrame setelah pengurutan
df_biologi

,Nama,Kelas,peringkatBiologi
53,KALYCA KHANSA FADILLA,XI A,1
0,AIRIN NURFAUZIAH MURSITO,X A,2
35,RAFI DZAKY UTAMA,X B,3
66,TARA TABINDA GALIH,XI A,4
4,CALISTA MICHELE AULIA,X A,5


In [ ]:
# Mengurutkan DataFrame berdasarkan peringkat Ekonomi
df_sortedEkonomi = df.sort_values(by='peringkatEkonomi', inplace = True)
kolom_ekonomi = ['Nama','Kelas', 'peringkatEkonomi']
df_ekonomi = df.head(5)[kolom_ekonomi]
df_allEkonomi = df.head(97)[kolom_ekonomi]

# Menampilkan DataFrame setelah pengurutan
df_ekonomi

,Nama,Kelas,peringkatEkonomi
0,AIRIN NURFAUZIAH MURSITO,X A,1
4,CALISTA MICHELE AULIA,X A,2
20,VIOLA AZIZAH,X A,3
8,KEYSHA FATCHU ISHTAMAR,X A,4
13,NOURA HONEY AYUNDA DIESMARA PUTERI,X A,5


In [ ]:
# Mengurutkan DataFrame berdasarkan peringkat Geografi
df_sortedGeografi = df.sort_values(by='peringkatGeografi', inplace = True)
kolom_geografi = ['Nama', 'Kelas','peringkatGeografi']
df_geografi = df.head(5)[kolom_geografi]
df_allGeografi = df.head(97)[kolom_geografi]

# Menampilkan DataFrame setelah pengurutan
df_geografi

,Nama,Kelas,peringkatGeografi
42,ANGELA PRATIWI PANCA PUTRI,XI A,1
65,SUSAN NADIVA MAZARINA,XI A,2
66,TARA TABINDA GALIH,XI A,3
59,OLIVIA CHAVY,XI A,4
67,YOLANDA AYU DANNA,XI A,5


In [ ]:
# Mengurutkan DataFrame berdasarkan peringkat TIK
df_sortedTIK = df.sort_values(by='peringkatTIK', inplace = True)
kolom_TIK = ['Nama','Kelas', 'peringkatTIK']
df_TIK = df.head(5)[kolom_TIK]
df_allTIK = df.head(97)[kolom_TIK]

# Menampilkan DataFrame setelah pengurutan
df_TIK

,Nama,Kelas,peringkatTIK
14,NUR AZIZAH LARASATI,X A,1
16,RENAISHA AZZAHRA NAYLA ARYO PUTRI,X A,2
4,CALISTA MICHELE AULIA,X A,3
53,KALYCA KHANSA FADILLA,XI A,4
13,NOURA HONEY AYUNDA DIESMARA PUTERI,X A,5


In [ ]:
# Menampilkan keseluruhan data
df['No'] = df['No'].rank(ascending=True, method='dense').astype(int)
df_sorted = df.sort_values(by='No', inplace = True)
df

,No,Nama,NIPD,JK,NISN,Kelas,nilaiHarian,nilaiUTS,MTK,fisika,...,V_ekonomi,V_geografi,V_TIK,peringkatMTK,peringkatFisika,peringkatKimia,peringkatBiologi,peringkatEkonomi,peringkatGeografi,peringkatTIK
0,1,AIRIN NURFAUZIAH MURSITO,232410001,P,81816310,X A,89,88.2,88,86,...,0.011163,0.010773,0.010082,9,1,1,2,1,8,78
1,2,ALYSHIA SAKHI,232410004,P,88170710,X A,82,83.4,80,83,...,0.010234,0.010313,0.010656,86,77,74,82,76,62,31
2,3,ANINDITA SAVIRA,232410005,P,83730957,X A,89,86.9,97,85,...,0.010800,0.010540,0.010891,3,18,18,29,6,34,7
3,4,AZKA ALTHAFUNNISA,232410006,P,71521082,X A,82,82.7,100,83,...,0.010423,0.010332,0.010752,8,73,77,77,41,59,16
4,5,CALISTA MICHELE AULIA,232410007,P,84251418,X A,86,86.9,83,85,...,0.011044,0.010658,0.010972,26,10,7,5,2,18,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,92,RAYHAN FAIZA NUGRAHA,222310056,P,72305569,XI B,88,85.3,84,85,...,0.010370,0.010616,0.010228,41,29,44,50,51,22,65
92,93,RIO TRIATMOJO,222310057,P,72471067,XI B,88,82.6,82,83,...,0.010487,0.010444,0.010343,37,20,32,33,31,40,55
93,94,SALMAN FIRDAUS,,,,XI B,85,83.9,82,85,...,0.010222,0.010181,0.010494,77,52,70,91,78,79,44
94,95,YUDHISTIRA PUTRAMARDJONO,,,,XI B,84,82.3,80,80,...,0.010185,0.010143,0.010045,87,91,85,89,86,84,85


##**Melakukan Input Data Kandidat ke Google Spreadsheet**

In [ ]:
new_sheet_name = 'Kandidat MTK'  # Ganti dengan nama sheet baru yang diinginkan
gc.create('Kandidat MTK')  # Membuat sheet baru dengan nama 'NewSheet'
sht2.add_worksheet(new_sheet_name, rows=len(df), cols=len(df.columns))  # Menambahkan worksheet baru
worksheet_new = sht2.worksheet(new_sheet_name)  # Mengakses worksheet baru

# Menulis DataFrame ke worksheet baru
worksheet_new.update([df_allMTK.columns.values.tolist()] + df_allMTK.values.tolist())

print(f"Data berhasil ditulis ke {new_sheet_name}")

Data berhasil ditulis ke Kandidat MTK


In [ ]:
new_sheet_name = 'Kandidat Fisika'  # Ganti dengan nama sheet baru yang diinginkan
gc.create('Kandidat Fisika')  # Membuat sheet baru dengan nama 'NewSheet'
sht2.add_worksheet(new_sheet_name, rows=len(df), cols=len(df.columns))  # Menambahkan worksheet baru
worksheet_new = sht2.worksheet(new_sheet_name)  # Mengakses worksheet baru

# Menulis DataFrame ke worksheet baru
worksheet_new.update([df_allFisika.columns.values.tolist()] + df_allFisika.values.tolist())

print(f"Data berhasil ditulis ke {new_sheet_name}")

Data berhasil ditulis ke Kandidat Fisika


In [ ]:
new_sheet_name = 'Kandidat Kimia'  # Ganti dengan nama sheet baru yang diinginkan
gc.create('Kandidat Kimia')  # Membuat sheet baru dengan nama 'NewSheet'
sht2.add_worksheet(new_sheet_name, rows=len(df), cols=len(df.columns))  # Menambahkan worksheet baru
worksheet_new = sht2.worksheet(new_sheet_name)  # Mengakses worksheet baru

# Menulis DataFrame ke worksheet baru
worksheet_new.update([df_allKimia.columns.values.tolist()] + df_allKimia.values.tolist())

print(f"Data berhasil ditulis ke {new_sheet_name}")

Data berhasil ditulis ke Kandidat Kimia


In [ ]:
new_sheet_name = 'Kandidat Biologi'  # Ganti dengan nama sheet baru yang diinginkan
gc.create('Kandidat Biologi')  # Membuat sheet baru dengan nama 'NewSheet'
sht2.add_worksheet(new_sheet_name, rows=len(df), cols=len(df.columns))  # Menambahkan worksheet baru
worksheet_new = sht2.worksheet(new_sheet_name)  # Mengakses worksheet baru

# Menulis DataFrame ke worksheet baru
worksheet_new.update([df_allBiologi.columns.values.tolist()] + df_allBiologi.values.tolist())

print(f"Data berhasil ditulis ke {new_sheet_name}")

Data berhasil ditulis ke Kandidat Biologi


In [ ]:
new_sheet_name = 'Kandidat Ekonomi'  # Ganti dengan nama sheet baru yang diinginkan
gc.create('Kandidat Ekonomi')  # Membuat sheet baru dengan nama 'NewSheet'
sht2.add_worksheet(new_sheet_name, rows=len(df), cols=len(df.columns))  # Menambahkan worksheet baru
worksheet_new = sht2.worksheet(new_sheet_name)  # Mengakses worksheet baru

# Menulis DataFrame ke worksheet baru
worksheet_new.update([df_allEkonomi.columns.values.tolist()] + df_allEkonomi.values.tolist())

print(f"Data berhasil ditulis ke {new_sheet_name}")

Data berhasil ditulis ke Kandidat Ekonomi


In [ ]:
new_sheet_name = 'Kandidat Geografi'  # Ganti dengan nama sheet baru yang diinginkan
gc.create('Kandidat Geografi')  # Membuat sheet baru dengan nama 'NewSheet'
sht2.add_worksheet(new_sheet_name, rows=len(df), cols=len(df.columns))  # Menambahkan worksheet baru
worksheet_new = sht2.worksheet(new_sheet_name)  # Mengakses worksheet baru

# Menulis DataFrame ke worksheet baru
worksheet_new.update([df_allGeografi.columns.values.tolist()] + df_allGeografi.values.tolist())

print(f"Data berhasil ditulis ke {new_sheet_name}")

Data berhasil ditulis ke Kandidat Geografi


In [ ]:
new_sheet_name = 'Kandidat TIK'  # Ganti dengan nama sheet baru yang diinginkan
gc.create('Kandidat TIK')  # Membuat sheet baru dengan nama 'NewSheet'
sht2.add_worksheet(new_sheet_name, rows=len(df), cols=len(df.columns))  # Menambahkan worksheet baru
worksheet_new = sht2.worksheet(new_sheet_name)  # Mengakses worksheet baru

# Menulis DataFrame ke worksheet baru
worksheet_new.update([df_allTIK.columns.values.tolist()] + df_allTIK.values.tolist())

print(f"Data berhasil ditulis ke {new_sheet_name}")

Data berhasil ditulis ke Kandidat TIK
